In [1]:
import os, fnmatch, glob, zipfile, shutil, warnings
import numpy as np
import pandas as pd
import xarray as xr
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
import calendar

warnings.filterwarnings("ignore")

In [2]:
# =========================
# Settings / paths
# =========================
dir_flux = "xxx"
fn_fluxlist = "fluxsite_info_metalist.xlsx"
out_dir = "xxx"

# Output filenames
fn_site_aridity = "site_aridity_index_yearly.csv"

# =========================
# Priestley–Taylor settings
# =========================
PT_ALPHA = 1.26  # default α

# =========================
# Physical constants
# =========================
es0 = 6.11           # hPa
T0 = 273.15          # K
Lv = 2.5e6           # J/kg
Rv = 461.0           # J/K/kg
Rd = 287.0           # J/K/kg
epson = Rd / Rv      # ~0.622
Cp = 1004.0          # J/K/kg
vK_const = 0.4       # von Karman constant
g = 9.8              # m/s^2

# =========================
# Columns & QC maps
# =========================
# FLUXNET / AmeriFlux / ICOS
pick_cols = [
    "TIMESTAMP_START","TIMESTAMP_END","P_F","P_F_QC","TA_F","TA_F_QC","PA_F","PA_F_QC",
    "SW_IN_F","SW_IN_F_QC","NETRAD","G_F_MDS","G_F_MDS_QC"
]
dic_QC1 = {
    "P_F":"P_F_QC","TA_F":"TA_F_QC","PA_F":"PA_F_QC","SW_IN_F":"SW_IN_F_QC",
    "G_F_MDS":"G_F_MDS_QC"
}
dic_QC2 = {
    "P_F":"P_F_QC","TA_F":"TA_F_QC","PA_F":"PA_F_QC",
    "SW_IN_F":"SW_IN_F_QC","G_flux":"G_F_MDS_QC"
}
# only these are replaced by QC3-cleaned values
dic_QC_flux = ["G_flux","NETRAD"]
var_to_QC = [
    "P_F","TA_F","PA_F","SW_IN_F","G_F_MDS"
]
drop_cols = [
    "P_F_QC","TA_F_QC","PA_F_QC","SW_IN_F_QC","G_F_MDS","G_F_MDS_QC","time_year"
]

# OzFlux
pick_cols_Ozflux = [
    "TIMESTAMP_START","TIMESTAMP_END","P_F","P_F_QC",
    "TA_F","TA_F_QC","PA_F","PA_F_QC","SW_IN_F","SW_IN_F_QC",
    "NETRAD","NETRAD_QC","G_flux","G_flux_QC",
]
dic_QC_Ozflux = {
    "P_F":"P_F_QC","TA_F":"TA_F_QC","PA_F":"PA_F_QC","SW_IN_F":"SW_IN_F_QC",
    "NETRAD":"NETRAD_QC","G_flux":"G_flux_QC"
}
var_QC_Ozflux_flux = ["G_flux","NETRAD",]
var_to_QC_Ozflux = ["P_F","TA_F","PA_F","SW_IN_F","NETRAD","G_flux"]
drop_cols_Ozflux = [
    "P_F_QC","TA_F_QC","PA_F_QC","SW_IN_F_QC","NETRAD_QC","G_flux_QC"
]

# LBA-ECO
pick_cols_LBAECO = [
    "TIMESTAMP_END","P_F","P_F_QC","TA_F","TA_F_QC","PA_F","PA_F_QC",
    "SW_IN_F","SW_IN_F_QC","NETRAD","NETRAD_QC","G_flux","G_flux_QC"
]
var_to_QC3_LBAECO = ["NETRAD","G_flux"]
drop_cols_LBAECO = [
    "P_F_QC","TA_F_QC","PA_F_QC","SW_IN_F_QC","NETRAD_QC","G_flux_QC"
]

In [3]:
# =========================
# Helper: outlier removal
# =========================
def remove_outlier(df, var):
    """
    Sliding ±7-day window, same-time-of-day (±2h) distribution.
    Flag outliers as |x - mean| > k*std, where k=5 for cold (TA_5day_mean<5C), else 3.
    Only enforce for flux-like vars (in dic_QC_flux). If not flux, drop when cold.
    """
    df_no_outlier = df.reset_index(drop=True).copy()
    outlier_mask = np.zeros(len(df_no_outlier), dtype=bool)

    if "hour_start" not in df_no_outlier.columns:
        df_no_outlier["hour_start"] = df_no_outlier["TIMESTAMP_START"].dt.hour

    ts_end = df_no_outlier["TIMESTAMP_END"].values
    hours  = df_no_outlier["hour_start"].values

    for idx in range(len(df_no_outlier)):
        center_time = ts_end[idx]
        h0 = hours[idx]
        window_start = center_time - np.timedelta64(7, "D")
        window_end   = center_time + np.timedelta64(7, "D")

        hour_diff = (hours - h0) % 24
        in_win = (ts_end >= window_start) & (ts_end <= window_end)
        same_time = (hour_diff <= 2)
        valid = in_win & same_time & (~pd.isna(df_no_outlier[var].values))

        window_values = df_no_outlier.loc[valid, var]
        if len(window_values) < 5:
            continue

        mean_val = window_values.mean()
        std_val  = window_values.std()

        tmask = (ts_end >= center_time - np.timedelta64(2, "D")) & (ts_end <= center_time + np.timedelta64(2, "D"))
        if "TA_low" in df_no_outlier.columns:
            TA_5day_mean = df_no_outlier.loc[tmask, "TA_low"].mean()
        else:
            TA_5day_mean = 10.0

        k = 5.0 if (TA_5day_mean < 5.0) else 3.0
        upper = mean_val + k * std_val
        lower = mean_val - k * std_val

        x = df_no_outlier.at[idx, var]
        if var in dic_QC_flux:
            if (x > upper) or (x < lower):
                outlier_mask[idx] = True
        else:
            if TA_5day_mean < 5.0:
                outlier_mask[idx] = True

    df_no_outlier.loc[outlier_mask, var] = np.nan
    return df_no_outlier

In [4]:
# =========================
# Extra helpers 
# =========================
def _days_in_year(y: int) -> int:
    return 366 if calendar.isleap(y) else 365


In [5]:
# =========================
# Worker
# =========================
def process_P_PET(args):
    site_ID, site_name, site_year_start, site_year_end, site_IGBP, site_src, z_measure, site_lat, site_lon, site_elv, canopy_h = args
    try:
        # -------------------------
        # Choose directories/patterns
        # -------------------------
        if site_src == "ICOS":
            zip_dir = "xxx/ICOS/warm_winter_2020_release_2022/"
            pattern = f"FLX_{site_ID}_*_FULLSET_*.zip"
        elif site_src == "AmeriFlux":
            zip_dir = "xxx/AmeriFlux/"
            pattern = f"AMF_{site_ID}_*_FULLSET_*.zip"
        elif site_src == "FLUXNET":
            zip_dir = "xxx/FLUXNET2015_zipped/"
            pattern = f"FLX_{site_ID}_*_FULLSET_*.zip"
        elif site_src == "Ozflux":
            nc_dir = "xxx/OzFlux/"
            pattern = site_name + "_L6.nc"
        elif site_src == "LBA-ECO":
            xlsx_dir = "xxx/LBA-ECO/"
            pattern = site_ID + "_CfluxBF.xlsx"
        else:
            print(f"[{site_ID}] Unknown source: {site_src}")
            return pd.DataFrame(columns=["site_ID","site_IGBP","year","P","PET","P_PET"])

        # -------------------------
        # Branch by source
        # -------------------------
        if site_src in ("ICOS", "AmeriFlux", "FLUXNET"):
            zip_path = None
            for entry in os.listdir(zip_dir):
                if fnmatch.fnmatch(entry, pattern):
                    zip_path = os.path.join(zip_dir, entry)
                    break
            if zip_path is None:
                print(f"[{site_ID}] No matching ZIP found.")
                return pd.DataFrame(columns=["site_ID","site_IGBP","year","P","PET","P_PET"])

            with zipfile.ZipFile(zip_path, "r") as zf:
                hh_names = [n for n in zf.namelist() if ("FULLSET_HH" in n and site_ID in n and n.endswith(".csv"))]
                hr_names = [n for n in zf.namelist() if ("FULLSET_HR" in n and site_ID in n and n.endswith(".csv"))]
                if not (hh_names or hr_names):
                    print(f"[{site_ID}] no FULLSET_HH/HR CSV inside {os.path.basename(zip_path)}")
                    return pd.DataFrame(columns=["site_ID","site_IGBP","year","P","PET","P_PET"])

                member_name = hh_names[0] if hh_names else hr_names[0]
                len_HH = 1 if "FULLSET_HH" in member_name else 0
                steps_per_day = 48 if len_HH else 24

                with zf.open(member_name) as fh:
                    flux_df = pd.read_csv(fh, usecols=lambda x: x in pick_cols)

            # -------------------------
            # Time & basic cleaning
            # -------------------------
            flux_df.replace(-9999, np.nan, inplace=True)
            flux_df["TIMESTAMP_START"] = pd.to_datetime(flux_df["TIMESTAMP_START"], format="%Y%m%d%H%M")
            flux_df["TIMESTAMP_END"]   = pd.to_datetime(flux_df["TIMESTAMP_END"],   format="%Y%m%d%H%M")
            flux_df["time_year"]       = flux_df["TIMESTAMP_START"].dt.year
            flux_df["time_DOY"]        = flux_df["TIMESTAMP_START"].dt.dayofyear

            # Keep early conversion (kPa -> hPa) used by some branches; PET path re-normalizes anyway
            flux_df["PA_F"] = flux_df["PA_F"] * 10.0

            # QC1
            for var in var_to_QC:
                qc_col = dic_QC1.get(var, None)
                if qc_col and (qc_col in flux_df.columns):
                    keep = flux_df[qc_col].isin([0, 1, 2]) | flux_df[qc_col].isna()
                    flux_df.loc[~keep, var] = np.nan

            # QC3 outlier removal
            site_data = pd.DataFrame({"TIMESTAMP_START": flux_df["TIMESTAMP_START"], "TIMESTAMP_END": flux_df["TIMESTAMP_END"]})
            flux_df["SW_IN_F"] = flux_df["SW_IN_F"]
            for var, qc in dic_QC2.items():
                if var not in flux_df.columns:
                    continue
                df_var = flux_df[["TIMESTAMP_START","TIMESTAMP_END","SW_IN_F",var]].copy()
                df_var["TA_low"]    = flux_df["TA_F"]
                df_var["TA_low_QC"] = flux_df.get("TA_F_QC", np.nan)
                if df_var["TA_low"].notna().any() and "TA_low_QC" in df_var.columns:
                    df_var["TA_low"] = df_var["TA_low"].where(df_var["TA_low_QC"].isin([0, 1, 2]), np.nan)
                df_var.drop(columns=["TA_low_QC"], inplace=True, errors="ignore")
                if var in dic_QC_flux:
                    df_no = remove_outlier(df_var, var)
                    df_no.drop(columns=["TA_low","hour_start"], inplace=True, errors="ignore")
                else:
                    df_no = df_var.drop(columns=["TA_low"])
                site_data = site_data.merge(df_no[["TIMESTAMP_START","TIMESTAMP_END",var]], on=["TIMESTAMP_START","TIMESTAMP_END"], how="left")

            # Replace QC3-cleaned targets
            KEYS = ["TIMESTAMP_START","TIMESTAMP_END"]
            targets = [c for c in dic_QC_flux if c in site_data.columns]
            if targets:
                site_data = site_data.drop_duplicates(subset=KEYS, keep="last")
                f_idx = flux_df.set_index(KEYS)
                s_idx = site_data.set_index(KEYS)
                miss = [c for c in targets if c not in f_idx.columns]
                if miss:
                    f_idx = f_idx.join(s_idx[miss], how="left")
                exist = [c for c in targets if c in f_idx.columns]
                if exist:
                    f_idx.update(s_idx[exist], overwrite=True)
                flux_df = f_idx.reset_index()


        elif site_src == "Ozflux":
            # --------------- OzFlux (NetCDF) ---------------
            flux_ds = xr.open_dataset(os.path.join(nc_dir, pattern))
            flux_df = pd.DataFrame(index=np.arange(flux_ds.dims["time"]))
            flux_df["TIMESTAMP_END"] = pd.to_datetime(np.squeeze(flux_ds["time"].values))
            flux_df["P_F"]       = np.squeeze(flux_ds["Precip"].values)
            flux_df["P_F_QC"]    = np.squeeze(flux_ds["Precip_QCFlag"].values)
            # flux_df["LE_flux"]   = np.squeeze(flux_ds["Fe"].values)
            # flux_df["LE_flux_QC"]= np.squeeze(flux_ds["Fe_QCFlag"].values)
            # flux_df["H_flux"]    = np.squeeze(flux_ds["Fh"].values)
            # flux_df["H_flux_QC"] = np.squeeze(flux_ds["Fh_QCFlag"].values)
            flux_df["TA_F"]      = np.squeeze(flux_ds["Ta"].values)
            flux_df["TA_F_QC"]   = np.squeeze(flux_ds["Ta_QCFlag"].values)
            flux_df["PA_F"]      = np.squeeze(flux_ds["ps"].values)
            flux_df["PA_F_QC"]   = np.squeeze(flux_ds["ps_QCFlag"].values)
            # if "GPP_LL" in flux_ds:
            #     flux_df["GPP"]    = np.squeeze(flux_ds["GPP_LL"].values)
            #     flux_df["GPP_QC"] = np.squeeze(flux_ds["GPP_LL_QCFlag"].values)
            # else:
            #     flux_df["GPP"]    = np.squeeze(flux_ds["GPP_SOLO"].values)
            #     flux_df["GPP_QC"] = np.squeeze(flux_ds["GPP_SOLO_QCFlag"].values)
            # flux_df["VPD_F"]     = np.squeeze(flux_ds["VPD"].values)
            # flux_df["VPD_F_QC"]  = np.squeeze(flux_ds["VPD_QCFlag"].values)
            flux_df["SW_IN_F"]   = np.squeeze(flux_ds["Fsd"].values)
            flux_df["SW_IN_F_QC"]= np.squeeze(flux_ds["Fsd_QCFlag"].values)
            # flux_df["WS_F"]      = np.squeeze(flux_ds["Ws"].values)
            # flux_df["WS_F_QC"]   = np.squeeze(flux_ds["Ws_QCFlag"].values)
            # flux_df["LW_IN_F"]   = np.squeeze(flux_ds["Fld"].values)
            # flux_df["LW_IN_F_QC"]= np.squeeze(flux_ds["Fld_QCFlag"].values)
            flux_df["NETRAD"]    = np.squeeze(flux_ds["Fn"].values)
            flux_df["NETRAD_QC"] = np.squeeze(flux_ds["Fn_QCFlag"].values)
            flux_df["G_flux"]    = np.squeeze(flux_ds["Fg"].values)
            flux_df["G_flux_QC"] = np.squeeze(flux_ds["Fg_QCFlag"].values)
            # flux_df["USTAR"]     = np.squeeze(flux_ds["ustar"].values)
            # flux_df["USTAR_QC"]  = np.squeeze(flux_ds["ustar_QCFlag"].values)
            flux_ds.close(); del flux_ds

            # units, timing
            flux_df["PA_F"] = flux_df["PA_F"] * 10.0
            # flux_df["VPD_F"] = flux_df["VPD_F"] * 10.0  # kPa -> hPa
            if (flux_df["TIMESTAMP_END"].iloc[1] - flux_df["TIMESTAMP_END"].iloc[0]) == pd.Timedelta(minutes=30):
                len_HH = 1
                flux_df["TIMESTAMP_START"] = flux_df["TIMESTAMP_END"] - pd.Timedelta(minutes=30)
            else:
                len_HH = 0
                flux_df["TIMESTAMP_START"] = flux_df["TIMESTAMP_END"] - pd.Timedelta(minutes=60)
            steps_per_day = 48 if len_HH else 24

            # QC1
            flux_df.replace(-9999, np.nan, inplace=True)
            for var in var_to_QC_Ozflux:
                qc = dic_QC_Ozflux[var]
                if qc in flux_df:
                    keep = flux_df[qc].isin([0, 10, 20, 410, 420, 430, 440, 450, 470, 570]) | flux_df[qc].isna()
                    flux_df.loc[~keep, var] = np.nan
            
            # QC3 outlier removal
            site_data = pd.DataFrame({"TIMESTAMP_START": flux_df["TIMESTAMP_START"], "TIMESTAMP_END": flux_df["TIMESTAMP_END"]})
            flux_df["time_year"] = flux_df["TIMESTAMP_START"].dt.year
            flux_df["time_DOY"]  = flux_df["TIMESTAMP_START"].dt.dayofyear
            flux_df["SW_IN_F"]   = flux_df["SW_IN_F"]
            for var, qc in dic_QC_Ozflux.items():
                if var not in flux_df.columns:
                    continue
                df_var = flux_df[["TIMESTAMP_START","TIMESTAMP_END","SW_IN_F",var]].copy()
                df_var["TA_low"] = flux_df["TA_F"]
                df_var["TA_low_QC"] = flux_df["TA_F_QC"]
                if df_var["TA_low"].notna().any():
                    df_var["TA_low"] = df_var["TA_low"].where(df_var["TA_low_QC"].isin([0, 10, 20, 410, 420, 430, 440, 450, 470, 570]), np.nan)
                df_var.drop(columns=["TA_low_QC"], inplace=True)
                if var in var_QC_Ozflux_flux:
                    df_no = remove_outlier(df_var, var)
                    df_no.drop(columns=["TA_low","hour_start"], inplace=True, errors="ignore")
                else:
                    df_no = df_var.drop(columns=["TA_low"])
                site_data = site_data.merge(df_no[["TIMESTAMP_START","TIMESTAMP_END",var]], on=["TIMESTAMP_START","TIMESTAMP_END"], how="left")

            # Replace QC3-cleaned targets
            KEYS = ["TIMESTAMP_START","TIMESTAMP_END"]
            targets = [c for c in var_QC_Ozflux_flux if c in site_data.columns]
            if targets:
                site_data = site_data.drop_duplicates(subset=KEYS, keep="last")
                f_idx = flux_df.set_index(KEYS); s_idx = site_data.set_index(KEYS)
                miss = [c for c in targets if c not in f_idx.columns]
                if miss: f_idx = f_idx.join(s_idx[miss], how="left")
                exist = [c for c in targets if c in f_idx.columns]
                if exist: f_idx.update(s_idx[exist], overwrite=True)
                flux_df = f_idx.reset_index()

        elif site_src == "LBA-ECO":
            # --------------- LBA-ECO (Excel) ---------------
            LBAECO_df = pd.read_excel(os.path.join(xlsx_dir, pattern), skiprows=[1])
            LBAECO_df["combined"] = LBAECO_df["Year_LBAMIP"].astype(int) * 1000 + LBAECO_df["DoY_LBAMIP"].astype(int)
            LBAECO_df["date"] = pd.to_datetime(LBAECO_df["combined"], format="%Y%j")
            LBAECO_df["datetime"] = LBAECO_df["date"] + pd.to_timedelta(LBAECO_df["Hour_LBAMIP"], unit="h")

            flux_df = pd.DataFrame(index=np.arange(len(LBAECO_df)))
            flux_df["TIMESTAMP_END"] = LBAECO_df["datetime"]
            flux_df["P_F"]           = LBAECO_df["prec"]
            # flux_df["LE_flux"]       = LBAECO_df["LE"]
            # flux_df["H_flux"]        = LBAECO_df["H"]
            flux_df["TA_F"]          = LBAECO_df["ta"]
            flux_df["PA_F"]          = LBAECO_df["press"]
            # flux_df["GPP_NT_VUT_REF"]= LBAECO_df["GEP_model"]
            # flux_df["GPP"]           = LBAECO_df["GEP_model"]
            # flux_df["VPD_F"]         = LBAECO_df["VPD"]
            flux_df["SW_IN_F"]       = LBAECO_df["rgs"]
            # flux_df["WS_F"]          = LBAECO_df["ws"]
            # flux_df["LW_IN_F"]       = LBAECO_df["rgl"]
            flux_df["NETRAD"]        = LBAECO_df["Rn"]
            flux_df["G_flux"]        = LBAECO_df["FG"]
            # flux_df["USTAR"]         = LBAECO_df["ust"]
            del LBAECO_df

            # units & timing
            flux_df["PA_F"]  = flux_df["PA_F"] * 10.0
            # flux_df["VPD_F"] = flux_df["VPD_F"] * 10.0  # kPa -> hPa
            if (flux_df["TIMESTAMP_END"].iloc[1] - flux_df["TIMESTAMP_END"].iloc[0]) == pd.Timedelta(minutes=30):
                len_HH = 1
                flux_df["TIMESTAMP_START"] = flux_df["TIMESTAMP_END"] - pd.Timedelta(minutes=30)
            else:
                len_HH = 0
                flux_df["TIMESTAMP_START"] = flux_df["TIMESTAMP_END"] - pd.Timedelta(minutes=60)
            steps_per_day = 48 if len_HH else 24

            # QC1
            flux_df.replace(-9999, np.nan, inplace=True)
            flux_df.loc[:, flux_df.select_dtypes(include=[np.number]).columns] = flux_df.select_dtypes(include=[np.number]).mask(flux_df.select_dtypes(include=[np.number]) < -9990)

            # QC3 outlier removal
            flux_df["time_year"] = flux_df["TIMESTAMP_START"].dt.year
            flux_df["time_DOY"]  = flux_df["TIMESTAMP_START"].dt.dayofyear
            site_data = pd.DataFrame({"TIMESTAMP_START": flux_df["TIMESTAMP_START"], "TIMESTAMP_END": flux_df["TIMESTAMP_END"], "SW_IN_F": flux_df["SW_IN_F"]})
            vars_present = [v for v in var_to_QC3_LBAECO if v in flux_df.columns]
            for var in vars_present:
                df_var = flux_df[["TIMESTAMP_START","TIMESTAMP_END","SW_IN_F",var]].copy()
                df_no = remove_outlier(df_var, var)
                df_no.drop(columns=["hour_start"], inplace=True, errors="ignore")
                site_data = site_data.merge(df_no[["TIMESTAMP_START","TIMESTAMP_END",var]], on=["TIMESTAMP_START","TIMESTAMP_END"], how="left")

            # Replace QC3-cleaned targets
            KEYS = ["TIMESTAMP_START","TIMESTAMP_END"]
            targets = [c for c in var_to_QC3_LBAECO if c in site_data.columns]
            if targets:
                site_data = site_data.drop_duplicates(subset=KEYS, keep="last")
                f_idx = flux_df.set_index(KEYS); s_idx = site_data.set_index(KEYS)
                miss = [c for c in targets if c not in f_idx.columns]
                if miss: f_idx = f_idx.join(s_idx[miss], how="left")
                exist = [c for c in targets if c in f_idx.columns]
                if exist: f_idx.update(s_idx[exist], overwrite=True)
                flux_df = f_idx.reset_index()

        print(f"finish all QC at site {site_ID}")

        # =========================
        # PET via Priestley–Taylor 
        # =========================
        P_Pa = flux_df["PA_F"].astype(float)  # hPa → Pa
        T_K = flux_df["TA_F"].astype(float) + T0
        e_s = 611.0 * np.exp((Lv/Rv) * (1.0/T0 - 1.0/T_K))  # Pa
        Delta = (Lv / Rv) * e_s / (T_K**2)               # Pa/K
        gamma = Cp * P_Pa / (epson * Lv)                 # Pa/K
        
        Rn = flux_df["NETRAD"].astype(float)
        Gv = flux_df["G_flux"].astype(float) if "G_flux" in flux_df.columns else np.nan
        Rn_G = np.where(np.isfinite(Gv), Rn - Gv, Rn)

        precip = flux_df["P_F"].astype(float)

        print(f"precip , min = {np.nanmin(precip)}, median = {np.nanmedian(precip)}, mean = {np.nanmean(precip)}, max = {np.nanmax(precip)}")
        print(f"Rn_G , min = {np.nanmin(Rn_G)}, median = {np.nanmedian(Rn_G)}, mean = {np.nanmean(Rn_G)}, max = {np.nanmax(Rn_G)}")
        print(f"s , min = {np.nanmin(Delta)}, median = {np.nanmedian(Delta)}, mean = {np.nanmean(Delta)}, max = {np.nanmax(Delta)}")
        print(f"gamma , min = {np.nanmin(gamma)}, median = {np.nanmedian(gamma)}, mean = {np.nanmean(gamma)}, max = {np.nanmax(gamma)}")
        # PM (energy form) -> W m^-2

        PET_Wm2 = PT_ALPHA * (Delta / (Delta + gamma)) * Rn_G
        PET_Wm2 = np.where(np.isfinite(PET_Wm2), PET_Wm2, np.nan)
        flux_df["PET_Wm2"] = PET_Wm2

        if 'steps_per_day' not in locals():
            dt_sec = (flux_df["TIMESTAMP_END"].iloc[1] - flux_df["TIMESTAMP_END"].iloc[0]).total_seconds()
            steps_per_day = 48 if abs(dt_sec - 1800) < 1 else 24
            
        flux_df["PET_mm"] = flux_df["PET_Wm2"]*60*60*(24/steps_per_day)/Lv # mm/30mins or mm/hr

        # =========================
        # Daily aggregation
        # =========================
        if 'steps_per_day' not in locals():
            dt_sec = (flux_df["TIMESTAMP_END"].iloc[1] - flux_df["TIMESTAMP_END"].iloc[0]).total_seconds()
            steps_per_day = 48 if abs(dt_sec - 1800) < 1 else 24

        daily = (
            flux_df.set_index('TIMESTAMP_START')
                   .resample('D')
                   .agg({
                        'P_F':     (lambda x: x.sum(min_count=int(0.8 * steps_per_day), skipna=True)),
                        'PET_mm':  (lambda x: x.sum(min_count=int(0.8 * steps_per_day), skipna=True))
                   })
                   .rename(columns={'P_F': 'P_mm', 'PET_mm': 'PET_sum_mm'})
        )
        # 'PET_Wm2': (lambda x: x.mean(skipna=True) if x.count() >= max(10, int(0.5 * steps_per_day)) else np.nan)
        # daily["PET_mean_mm"] = daily["PET_Wm2"] * 86400.0 / Lv   # W m^-2 → mm day^-1
        daily["year"] = daily.index.year

        print(f"get daily PET and precip. at site {site_ID}: {daily[['P_mm','PET_sum_mm']].head(20)}")

        # =========================
        # Only (≥50%) recorded years
        # =========================
        rows = []
        for y, g in daily.groupby("year"):
            nd = _days_in_year(int(y))              # 365 or 366
            req = int(np.ceil(0.8 * nd))           # required valid days

            # day is valid if P and PET exist and PET > 0
            mask = g["P_mm"].notna() & g["PET_sum_mm"].notna() & (g["PET_sum_mm"] > 0)
            valid_days = int(mask.sum())
            if valid_days < req:
                rows.append({
                    "site_ID": site_ID,
                    "site_IGBP": site_IGBP,
                    "year": int(y),
                    "P": np.nan,
                    "PET": np.nan,
                    "P_PET": np.nan
                })
                continue

            P_sum    = float(g.loc[mask, "P_mm"].sum())    # mm/year
            PET_sum  = float(g.loc[mask, "PET_sum_mm"].sum())  # mm/year (sum of daily)
        
            rows.append({
                "site_ID": site_ID,
                "site_IGBP": site_IGBP,
                "year": int(y),
                "P": P_sum,
                "PET": PET_sum,
                "P_PET": P_sum / PET_sum
            })

        print(f"get yearly PET and precip. at site {site_ID}: {rows}")

        if not rows:
            return pd.DataFrame(columns=["site_ID","site_IGBP","year","P","PET","P_PET"])
        return pd.DataFrame(rows)

    except Exception as e:
        print(f"[{site_ID}] Error: {e}")
        return pd.DataFrame(columns=["site_ID","site_IGBP","year","P","PET","P_PET"])


In [6]:
# =========================
# Main
# =========================
if __name__ == "__main__":
    # Load site list and filter
    df_sitelist = pd.read_excel(os.path.join(dir_flux, fn_fluxlist))
    forest_site_mask = (
        (df_sitelist["IGBP"].isin(["EBF","ENF","DBF","DNF","MF"])) &
        ((df_sitelist["year_end"] - df_sitelist["year_start"] + 1) >= 3)
    )
    df_forest_sitelist = df_sitelist[forest_site_mask].reset_index(drop=True)
    total_site = len(df_forest_sitelist)
    print("total site =", total_site)

    # Build process args
    site_info_list = []
    for st in range(total_site):  # change to total_site to run all
        row = df_forest_sitelist.iloc[st]
        site_info_list.append((
            row["site_ID"],
            row["SITE_NAME"],
            int(row["year_start"]),
            int(row["year_end"]),
            row["IGBP"],
            row["source"],
            row["measure_h"],
            row["LOCATION_LAT"],
            row["LOCATION_LONG"],
            row["LOCATION_ELEV"],
            row.get("canopy_h", np.nan)
        ))
    print(f"Appended all {len(site_info_list)} sites!")

    # Multiprocessing
    from os import cpu_count
    max_workers = min(8, (cpu_count() or 4))
    results = []
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(process_P_PET, args): args[0] for args in site_info_list}
        for fut in tqdm(as_completed(futures), total=len(futures), desc="Processing sites"):
            site_ID = futures[fut]
            try:
                df_out = fut.result()
                if df_out is not None and not df_out.empty:
                    results.append(df_out)
            except Exception as e:
                print(f"Error processing site {site_ID}: {e}")

    # Save outputs
    if results:
        df_site_aridity = pd.concat(results, ignore_index=True)
        df_site_aridity = df_site_aridity.sort_values(["site_ID","year"]).reset_index(drop=True)
        os.makedirs(out_dir, exist_ok=True)
        df_site_aridity.to_csv(os.path.join(out_dir, fn_site_aridity), index=False)
        print(f"Saved {fn_site_aridity} with {len(df_site_aridity)} rows.")
    else:
        print("No per-site aridity rows to save.")


total site = 177
Appended all 177 sites!


Processing sites:   0%|                                                                            | 0/177 [00:00<?, ?it/s]

finish all QC at site AR-SLu
precip , min = 0.0, median = 0.0, mean = 0.02060487062404871, max = 23.6
Rn_G , min = -122.699, median = -20.5101, mean = 119.30238621645084, max = 874.365
s , min = 29.52184944148479, median = 151.9413207168163, mean = 161.9160999146429, max = 554.0873267882124
gamma , min = 0.6024196881951219, median = 0.61494711743554, mean = 0.6151608413761411, max = 0.6308870132404181
get daily PET and precip. at site AR-SLu:                    P_mm  PET_sum_mm
TIMESTAMP_START                    
2009-01-01        3.072         NaN
2009-01-02        0.694         NaN
2009-01-03        0.000         NaN
2009-01-04        0.000         NaN
2009-01-05        0.986         NaN
2009-01-06        6.060         NaN
2009-01-07        0.000         NaN
2009-01-08        0.044         NaN
2009-01-09        0.000         NaN
2009-01-10        0.000         NaN
2009-01-11        5.942         NaN
2009-01-12       14.582         NaN
2009-01-13        0.000         NaN
2009-01-14   

Processing sites:   1%|▍                                                                   | 1/177 [00:01<03:28,  1.18s/it]

finish all QC at site AR-Vir
precip , min = 0.0, median = 0.0, mean = 0.08090109514031486, max = 8.346
Rn_G , min = -135.992, median = -12.00925, mean = 123.38748276092134, max = 908.468
s , min = 41.8640371638852, median = 169.24134877562932, mean = 172.4872634965587, max = 375.32840869252624
gamma , min = 0.6345059050034844, median = 0.6441691846132402, mean = 0.6446428339213933, max = 0.6588253737142856
get daily PET and precip. at site AR-Vir:                    P_mm  PET_sum_mm
TIMESTAMP_START                    
2009-01-01        0.000         NaN
2009-01-02        6.912         NaN
2009-01-03        0.324         NaN
2009-01-04        0.000         NaN
2009-01-05        0.000         NaN
2009-01-06        0.000         NaN
2009-01-07        0.000         NaN
2009-01-08        1.104         NaN
2009-01-09        7.938         NaN
2009-01-10       12.150         NaN
2009-01-11        0.852         NaN
2009-01-12        1.146         NaN
2009-01-13       10.434         NaN
2009-01-

...

finish all QC at site AU-Wom
precip , min = 0.0, median = 0.0, mean = 0.04883748618514731, max = 19.8
Rn_G , min = -253.19985626799252, median = 8.122748037163106, mean = 114.28216873698413, max = 1073.6156836800496
s , min = 31.94955155734734, median = 82.02320847002363, mean = 95.65084758416234, max = 382.5847020627236
gamma , min = 0.5836328663620906, median = 0.6013855947119164, mean = 0.6013109297228744, max = 0.6151579936774912
get daily PET and precip. at site AU-Wom:                      P_mm  PET_sum_mm
TIMESTAMP_START                      
2010-01-20            NaN         NaN
2010-01-21       0.000000    9.608918
2010-01-22       0.000000    3.092937
2010-01-23       0.000000    7.756471
2010-01-24       0.000000    9.758298
2010-01-25       0.000000    9.256168
2010-01-26       0.000000    9.254649
2010-01-27       0.000000   10.122610
2010-01-28       0.000000    9.409742
2010-01-29       0.000000    9.073901
2010-01-30       0.000000    8.754244
2010-01-31       0.000000 

Processing sites:  99%|█████████████████████████████████████████████████████████████████▋| 176/177 [17:54<00:42, 42.07s/it]

finish all QC at site AU-Cow
precip , min = 0.0, median = 0.0, mean = 0.1705211008117337, max = 85.8
Rn_G , min = -154.45248549999997, median = -0.3805616300000292, mean = 93.62076589322554, max = 1001.2304900207108
s , min = 44.40970454240359, median = 189.86387645215703, mean = 193.94778576717704, max = 662.0417466064738
gamma , min = 0.6003968502645296, median = 0.6452858159219512, mean = 0.6450323411162766, max = 0.6534441268181185
get daily PET and precip. at site AU-Cow:                  P_mm  PET_sum_mm
TIMESTAMP_START                  
2009-01-01        NaN         NaN
2009-01-02        0.0    8.684522
2009-01-03        0.0    7.205662
2009-01-04        0.0    5.707830
2009-01-05        0.0    2.423459
2009-01-06        0.0    6.614534
2009-01-07        0.1    5.086857
2009-01-08        0.0    8.216601
2009-01-09        0.0    0.889484
2009-01-10        2.0    2.992086
2009-01-11        1.2    0.995253
2009-01-12        0.0    1.788904
2009-01-13        0.0    7.789615
2009-01-

Processing sites: 100%|██████████████████████████████████████████████████████████████████| 177/177 [31:17<00:00, 10.61s/it]

Saved site_aridity_index_yearly.csv with 1563 rows.


In [19]:
# =========================
# Main
# =========================
if __name__ == "__main__":
    # Load site list and filter
    df_sitelist = pd.read_excel(os.path.join(dir_flux, fn_fluxlist))
    forest_site_mask = (
        (df_sitelist["IGBP"].isin(["EBF","ENF","DBF","DNF","MF"])) &
        ((df_sitelist["year_end"] - df_sitelist["year_start"] + 1) >= 3)
    )
    df_forest_sitelist = df_sitelist[forest_site_mask].reset_index(drop=True)
    total_site = len(df_forest_sitelist)
    print("total site =", total_site)

    # Build process args
    site_info_list = []
    for st in range(3):  # change to total_site to run all
        row = df_forest_sitelist.iloc[st]
        site_info_list.append((
            row["site_ID"],
            row["SITE_NAME"],
            int(row["year_start"]),
            int(row["year_end"]),
            row["IGBP"],
            row["source"],
            row["measure_h"],
            row["LOCATION_LAT"],
            row["LOCATION_LONG"],
            row["LOCATION_ELEV"],
            row.get("canopy_h", np.nan)
        ))
    print(f"Appended all {len(site_info_list)} sites!")

    # Multiprocessing
    from os import cpu_count
    max_workers = min(8, (cpu_count() or 4))
    results = []
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(process_P_PET, args): args[0] for args in site_info_list}
        for fut in tqdm(as_completed(futures), total=len(futures), desc="Processing sites"):
            site_ID = futures[fut]
            try:
                df_out = fut.result()
                if df_out is not None and not df_out.empty:
                    results.append(df_out)
            except Exception as e:
                print(f"Error processing site {site_ID}: {e}")

    # Save outputs
    if results:
        df_site_aridity = pd.concat(results, ignore_index=True)
        df_site_aridity = df_site_aridity.sort_values(["site_ID","year"]).reset_index(drop=True)
        os.makedirs(out_dir, exist_ok=True)
        df_site_aridity.to_csv(os.path.join(out_dir, fn_site_aridity), index=False)
        print(f"Saved {fn_site_aridity} with {len(df_site_aridity)} rows.")
    else:
        print("No per-site aridity rows to save.")


total site = 177
Appended all 3 sites!


Processing sites:   0%|                                                                              | 0/3 [00:00<?, ?it/s]

finish all QC at site AR-SLu
precip = 0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
52555    0.0
52556    0.0
52557    0.0
52558    0.0
52559    0.0
Name: P_F, Length: 52560, dtype: float64, min = 0.0, median = 0.0, mean = 0.02060487062404871, max = 23.6
Rn_G = [nan nan nan ... nan nan nan], min = -122.699, median = -20.5101, mean = 119.30238621645084, max = 874.365
s = 0        148.277082
1        148.611547
2        148.946669
3        149.274047
4        149.610471
            ...    
52555    305.399571
52556    295.484732
52557    285.849489
52558    276.487266
52559    267.391604
Name: TA_F, Length: 52560, dtype: float64, min = 29.52184944148479, median = 151.9413207168163, mean = 161.9160999146429, max = 554.0873267882124
gamma = 0        0.615289
1        0.615199
2        0.615115
3        0.615025
4        0.614941
           ...   
52555    0.613463
52556    0.613541
52557    0.613618
52558    0.613689
52559    0.613767
Name: PA_F, Length: 525

Processing sites:  33%|███████████████████████▎                                              | 1/3 [00:01<00:02,  1.09s/it]

finish all QC at site AR-Vir
precip = 0        0.000
1        0.000
2        0.000
3        0.000
4        0.000
         ...  
70123    0.397
70124    0.397
70125    0.397
70126    0.397
70127    0.397
Name: P_F, Length: 70128, dtype: float64, min = 0.0, median = 0.0, mean = 0.08090109514031486, max = 8.346
Rn_G = [nan nan nan ... nan nan nan], min = -135.992, median = -12.00925, mean = 123.38748276092134, max = 908.468
s = 0        216.798293
1        213.553257
2        210.350467
3        207.189463
4        204.069788
            ...    
70123    220.404209
70124    215.060782
70125    209.841458
70126    204.721403
70127    199.721016
Name: TA_F, Length: 70128, dtype: float64, min = 41.8640371638852, median = 169.24134877562932, mean = 172.4872634965587, max = 375.32840869252624
gamma = 0        0.642989
1        0.642911
2        0.642834
3        0.642756
4        0.642679
           ...   
70123    0.640744
70124    0.640918
70125    0.641086
70126    0.641253
70127    0.64142

Processing sites:  67%|██████████████████████████████████████████████▋                       | 2/3 [00:01<00:00,  1.62it/s]

finish all QC at site AU-Cum
precip = 0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
175242    2.8
175243    0.6
175244    0.0
175245    0.0
175246    0.2
Name: P_F, Length: 175247, dtype: float64, min = 0.0, median = 0.0, mean = 0.0516356913385108, max = 30.0
Rn_G = [ -6.65560936 -14.22393038 -18.22571866 ...   6.81018101   9.33144338
  11.72907866], min = -259.5208389771634, median = 3.671511888090052, mean = 123.01402951875583, max = 1187.2164981969981
s = 0         159.437191
1         159.103355
2         156.661734
3         154.983087
4         153.725331
             ...    
175242    137.818097
175243    135.017259
175244    132.011383
175245    130.111545
175246    129.149721
Name: TA_F, Length: 175247, dtype: float64, min = 39.44956495058238, median = 134.70902259313982, mean = 143.10406517564968, max = 577.4164609198039
gamma = 0         0.650622
1         0.650178
2         0.649708
3         0.649533
4         0.649323
            ... 

Processing sites: 100%|█████████████████████████████████████████████████████████████████████| 3/3 [12:33<00:00, 251.08s/it]

Saved site_aridity_index_yearly.csv with 12 rows.


In [20]:
# =========================
# Main
# =========================
if __name__ == "__main__":
    # Load site list and filter
    df_sitelist = pd.read_excel(os.path.join(dir_flux, fn_fluxlist))
    forest_site_mask = (
        (df_sitelist["IGBP"].isin(["EBF","ENF","DBF","DNF","MF"])) &
        ((df_sitelist["year_end"] - df_sitelist["year_start"] + 1) >= 3)
    )
    df_forest_sitelist = df_sitelist[forest_site_mask].reset_index(drop=True)
    total_site = len(df_forest_sitelist)
    print("total site =", total_site)

    # Build process args
    site_info_list = []
    for st in range(152,153,1):  # change to total_site to run all
        row = df_forest_sitelist.iloc[st]
        site_info_list.append((
            row["site_ID"],
            row["SITE_NAME"],
            int(row["year_start"]),
            int(row["year_end"]),
            row["IGBP"],
            row["source"],
            row["measure_h"],
            row["LOCATION_LAT"],
            row["LOCATION_LONG"],
            row["LOCATION_ELEV"],
            row.get("canopy_h", np.nan)
        ))
    print(f"Appended all {len(site_info_list)} sites!")

    # Multiprocessing
    from os import cpu_count
    max_workers = min(8, (cpu_count() or 4))
    results = []
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(process_P_PET, args): args[0] for args in site_info_list}
        for fut in tqdm(as_completed(futures), total=len(futures), desc="Processing sites"):
            site_ID = futures[fut]
            try:
                df_out = fut.result()
                if df_out is not None and not df_out.empty:
                    results.append(df_out)
            except Exception as e:
                print(f"Error processing site {site_ID}: {e}")

    # Save outputs
    if results:
        df_site_aridity = pd.concat(results, ignore_index=True)
        df_site_aridity = df_site_aridity.sort_values(["site_ID","year"]).reset_index(drop=True)
        os.makedirs(out_dir, exist_ok=True)
        df_site_aridity.to_csv(os.path.join(out_dir, fn_site_aridity), index=False)
        print(f"Saved {fn_site_aridity} with {len(df_site_aridity)} rows.")
    else:
        print("No per-site aridity rows to save.")


total site = 177
Appended all 1 sites!


Processing sites:   0%|                                                                              | 0/1 [00:00<?, ?it/s]

finish all QC at site BAN
precip = 0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
35035   NaN
35036   NaN
35037   NaN
35038   NaN
35039   NaN
Name: P_F, Length: 35040, dtype: float64, min = 0.0, median = 0.0, mean = 0.1965328947368419, max = 54.35
Rn_G = [nan nan nan ... nan nan nan], min = -152.6211, median = -10.40249999999999, mean = 135.3669317277142, max = 904.825
s = 0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
35035   NaN
35036   NaN
35037   NaN
35038   NaN
35039   NaN
Name: TA_F, Length: 35040, dtype: float64, min = 122.66110918115048, median = 203.58501066660642, mean = 214.94704810147275, max = 371.6531569584942
gamma = 0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
35035   NaN
35036   NaN
35037   NaN
35038   NaN
35039   NaN
Name: PA_F, Length: 35040, dtype: float64, min = 0.6342549693714286, median = 0.6392549747289198, mean = 0.6392319090798967, max = 0.6446613796961665
get daily PET and precip. at

Processing sites: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [01:34<00:00, 94.04s/it]

Saved site_aridity_index_yearly.csv with 2 rows.
